# Sentiment Analysis Techniques

In [1]:
import os
import sys


# This adds the 'src' directory to the Python path.
src_rel_path = "../src"
src_abs_path = os.path.abspath(src_rel_path)

if src_abs_path not in sys.path:
    sys.path.append(src_abs_path)
    print(f"Added '{src_abs_path}' to sys.path")
else:
    print(f"'{src_abs_path}' is already in sys.path")

import database as db

Added '/Users/shubxam/Code/Nifty-500-Live-Sentiment-Analysis/src' to sys.path


In [3]:
conn = db.DatabaseManager()

In [4]:
articles_df = conn.get_articles()

In [5]:
articles_df

,ticker,headline,date_posted,source,article_link,negative_sentiment,positive_sentiment,neutral_sentiment,compound_sentiment,created_at
0,ADANIENT,"Top Gainers and Losers today on 28 August, 202...",2024-08-29 01:30:14,Livemint,https://www.livemint.com/market/stock-market-n...,0.999884,0.000069,0.000048,-0.000021,2024-08-31 01:30:45.048
1,ADANIENT,"6 Adani stocks, LIC, and 3 PSUs slip below 20 ...",2024-08-29 01:30:14,The Economic Times,https://m.economictimes.com/markets/stocks/new...,0.004788,0.994831,0.000382,-0.994449,2024-08-31 01:30:45.048
2,ADANIENT,"Adani Group invests Rs 2,000 crore in Dharavi ...",2024-08-30 11:30:14,Business Standard,https://www.business-standard.com/companies/ne...,0.999818,0.000068,0.000114,0.000047,2024-08-31 01:30:45.048
3,ADANIENT,Adani’s airport business to undergo de-merger ...,2024-08-30 01:30:14,Times of India,https://timesofindia.indiatimes.com/business/i...,0.999845,0.000074,0.000081,0.000007,2024-08-31 01:30:45.048
4,ADANIENT,Adani Ports acquires 80% stake in global OSV o...,2024-08-30 15:30:14,The Financial Express,https://www.financialexpress.com/business/indu...,0.999771,0.000061,0.000168,0.000107,2024-08-31 01:30:45.048
...,...,...,...,...,...,...,...,...,...,...
14640,WIPRO,"Coforge, Wipro, Infosys lead Nifty IT rebound ...",2025-04-11 11:35:27,Moneycontrol,https://www.moneycontrol.com/news/business/mar...,NaN,NaN,NaN,NaN,2025-04-12 11:35:30.738
14641,WIPRO,"Infosys, Wipro ADRs fall by 3% after Accenture...",2025-03-22 11:35:27,The Economic Times,https://m.economictimes.com/markets/stocks/new...,NaN,NaN,NaN,NaN,2025-04-12 11:35:30.738
14642,WIPRO,Wipro (WIT) Projected to Post Earnings on Wedn...,2025-04-12 03:35:27,MarketBeat,https://www.marketbeat.com/instant-alerts/wipr...,NaN,NaN,NaN,NaN,2025-04-12 11:35:30.738
14643,WIPRO,"Infosys, HCL Tech, Wipro Surge Up To 2% As IT ...",2025-04-11 11:35:27,Republic World,https://www.republicworld.com/business/infosys...,NaN,NaN,NaN,NaN,2025-04-12 11:35:30.738


## External APIs